In [1]:
import torch
from transformers import ViTFeatureExtractor, ViTForImageClassification, AdamW
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, Subset
from sklearn.model_selection import train_test_split
from tqdm import tqdm

2023-08-24 09:08:32.755682: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-24 09:08:44.208245: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
ls

dataverse/  data.zip  Untitled.ipynb  unzip_script.py  vit_train.ipynb


# MobileNet

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

In [3]:
# Define transformations for data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset
train_dataset = torchvision.datasets.ImageFolder('./dataverse/train', transform=transform)
test_dataset = torchvision.datasets.ImageFolder('./dataverse/test', transform=transform)
val_dataset = torchvision.datasets.ImageFolder('./dataverse/val', transform=transform)

In [15]:
# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Load MobileNet model
model = models.mobilenet_v2(pretrained=True)
num_classes = len(train_dataset.classes)
model.classifier[1] = nn.Linear(1280, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib64/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/usr/local/lib64/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

### Training Loop

In [16]:
import numpy as np
# Lists to store training and validation losses, accuracy, and AUC-ROC
train_losses = []
val_losses = []
val_accuracies = []
val_aurocs = []

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    
    # Calculate average training loss for the epoch
    train_loss /= len(train_loader.dataset)
    train_losses.append(train_loss)
    
    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_predictions.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate average validation loss for the epoch
    val_loss /= len(val_loader.dataset)
    val_losses.append(val_loss)
    
    # Calculate validation accuracy
    val_accuracy = 100 * correct / total
    val_accuracies.append(val_accuracy)
    
    # Calculate validation AUC-ROC
    all_predictions_softmax = np.array([np.exp(pred) / np.sum(np.exp(pred)) for pred in all_predictions])
    val_auroc = roc_auc_score(all_labels, all_predictions_softmax, multi_class='ovr', average='macro')
    val_aurocs.append(val_auroc)
    
    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"Train Loss: {train_loss:.4f}, "
          f"Val Loss: {val_loss:.4f}, "
          f"Val Acc: {val_accuracy:.2f}%, "
          f"Val AUC-ROC: {val_auroc:.4f}")

Epoch [1/5] - Train Loss: 0.3367, Val Loss: 0.1898, Val Acc: 93.68%, Val AUC-ROC: 0.9946
Epoch [2/5] - Train Loss: 0.2128, Val Loss: 0.1820, Val Acc: 93.24%, Val AUC-ROC: 0.9958
Epoch [3/5] - Train Loss: 0.1773, Val Loss: 0.1426, Val Acc: 94.93%, Val AUC-ROC: 0.9965
Epoch [4/5] - Train Loss: 0.1649, Val Loss: 0.1317, Val Acc: 95.27%, Val AUC-ROC: 0.9969
Epoch [5/5] - Train Loss: 0.1371, Val Loss: 0.2198, Val Acc: 92.64%, Val AUC-ROC: 0.9947


In [17]:
# Save the trained model
model_filename = 'mobilenet_model.pth'
torch.save(model.state_dict(), model_filename)

print(f"Model saved as {model_filename}")

Model saved as mobilenet_model.pth


In [ ]:
# Load the saved model
loaded_model = models.mobilenet_v2(pretrained=False)  # Create a new instance of the model
loaded_model.classifier[1] = nn.Linear(1280, num_classes)  # Adjust classifier layer

loaded_model.load_state_dict(torch.load(model_filename))
loaded_model.eval()  # Set the model to evaluation mode

print("Model loaded successfully")
